In [2]:
import os
import warnings
import math
import itertools
import numpy as np
import pandas as pd
from datetime import datetime
from typing import List, Dict, Tuple

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns
import shap

# shap.enable_js()  # Removed because shap does not have enable_js()
warnings.filterwarnings("ignore")

/Users/dela/Developer/thesis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = 'datasets/cleaned/merged_fx_dataset.csv'
TARGET_COL = "Buying"          # main target
DATE_COL = "Date"
WEEK_RULE = "W-WED"           # weekly grid (Wednesdays)
LAGS = list(range(1, 9))  # AR/exogenous lags: 1..8 weeks
HORIZONS = [1, 4]            # 1-week and 4-week ahead
N_SPLITS = 5                 # expanding CV folds
RANDOM_SEED = 42

In [4]:
df = pd.read_csv(file_path)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3703 entries, 0 to 3702
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  3703 non-null   object 
 1   Buying                3703 non-null   float64
 2   Selling               3701 non-null   float64
 3   MidRate               3701 non-null   float64
 4   MPR                   3703 non-null   float64
 5   FoodInflation         3703 non-null   float64
 6   GhInflationRate       3703 non-null   float64
 7   NonFoodInflation      3703 non-null   float64
 8   BrentOil              3703 non-null   float64
 9   Cocoa                 3703 non-null   float64
 10  Gold                  3703 non-null   float64
 11  GhInterestRate        3703 non-null   float64
 12  NetForeignAssets      3703 non-null   float64
 13  NIR                   3703 non-null   float64
 14  Imports               3703 non-null   float64
 15  Exports              

In [6]:
df.head()

,Date,Buying,Selling,MidRate,MPR,FoodInflation,GhInflationRate,NonFoodInflation,BrentOil,Cocoa,...,GhCompositeIndex,FXDeposits,InterbankWeightedAvg,T-bill-182,T-bill-91,PrivateSectorCredit,TradeBalance,USGDP,USInflationRate,USInterestRate
0,2022-12-01,13.0973,13.1105,13.1039,27.0,59.7,54.1,49.9,81.34,2538.57,...,0.00,45124.35,25.51,36.23,35.48,63753.45,450.75,22249.459,116.977,3.83
1,2022-11-30,13.0978,13.1110,13.1044,27.0,55.3,50.3,46.5,90.38,2469.10,...,-6.16,66496.63,25.80,35.68,34.62,73744.05,444.36,22249.459,116.554,3.83
2,2022-11-29,13.0980,13.1112,13.1046,27.0,55.3,50.3,46.5,90.38,2469.10,...,-6.16,66496.63,25.80,35.68,34.62,73744.05,444.36,22249.459,116.554,3.83
3,2022-11-28,13.0982,13.1114,13.1048,27.0,55.3,50.3,46.5,90.38,2469.10,...,-6.16,66496.63,25.80,35.68,34.62,73744.05,444.36,22249.459,116.554,3.83
4,2022-11-25,13.0985,13.1117,13.1051,27.0,55.3,50.3,46.5,90.38,2469.10,...,-6.16,66496.63,25.80,35.68,34.62,73744.05,444.36,22249.459,116.554,3.83


In [7]:
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
df = df.sort_values(DATE_COL).reset_index(drop=True)
df = df.set_index(DATE_COL)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3703 entries, 2008-01-02 to 2022-12-01
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Buying                3703 non-null   float64
 1   Selling               3701 non-null   float64
 2   MidRate               3701 non-null   float64
 3   MPR                   3703 non-null   float64
 4   FoodInflation         3703 non-null   float64
 5   GhInflationRate       3703 non-null   float64
 6   NonFoodInflation      3703 non-null   float64
 7   BrentOil              3703 non-null   float64
 8   Cocoa                 3703 non-null   float64
 9   Gold                  3703 non-null   float64
 10  GhInterestRate        3703 non-null   float64
 11  NetForeignAssets      3703 non-null   float64
 12  NIR                   3703 non-null   float64
 13  Imports               3703 non-null   float64
 14  Exports               3703 non-null   float64
 15  GhG

In [9]:
df.columns

Index(['Buying', 'Selling', 'MidRate', 'MPR', 'FoodInflation',
       'GhInflationRate', 'NonFoodInflation', 'BrentOil', 'Cocoa', 'Gold',
       'GhInterestRate', 'NetForeignAssets', 'NIR', 'Imports', 'Exports',
       'GhGDP', 'GhCompositeIndex', 'FXDeposits', 'InterbankWeightedAvg',
       'T-bill-182', 'T-bill-91', 'PrivateSectorCredit', 'TradeBalance',
       'USGDP', 'USInflationRate', 'USInterestRate'],
      dtype='object')

In [10]:
leak_cols = [c for c in ["Selling", "MidRate",
                         "InterbankWeightedAvg", "Imports", "Exports"] if c in df.columns]
df = df.drop(columns=leak_cols, errors="ignore")

In [11]:
if (df[TARGET_COL] == 0).any():
    df.loc[df[TARGET_COL] == 0, TARGET_COL] = np.nan
df[TARGET_COL] = df[TARGET_COL].ffill().bfill()

In [12]:
df = df.select_dtypes(include=[np.number]).join(
    df.index.to_series(name=DATE_COL))
df = df.set_index(DATE_COL)

print("Shape (daily/mixed):", df.shape)
df.head()

Shape (daily/mixed): (3703, 21)


,Buying,MPR,FoodInflation,GhInflationRate,NonFoodInflation,BrentOil,Cocoa,Gold,GhInterestRate,NetForeignAssets,...,GhGDP,GhCompositeIndex,FXDeposits,T-bill-182,T-bill-91,PrivateSectorCredit,TradeBalance,USGDP,USInflationRate,USInterestRate
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,0.9545,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.11
2008-01-03,0.9545,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.25
2008-01-04,0.9543,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.18
2008-01-07,0.9551,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.27
2008-01-08,0.9576,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.27


In [13]:
df_weekly = df.copy().ffill().resample(
    WEEK_RULE).last().dropna(subset=[TARGET_COL])
print("Shape (weekly):", df_weekly.shape)
df_weekly.head()

Shape (weekly): (780, 21)


,Buying,MPR,FoodInflation,GhInflationRate,NonFoodInflation,BrentOil,Cocoa,Gold,GhInterestRate,NetForeignAssets,...,GhGDP,GhCompositeIndex,FXDeposits,T-bill-182,T-bill-91,PrivateSectorCredit,TradeBalance,USGDP,USInflationRate,USInterestRate
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,0.9545,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.11
2008-01-09,0.9574,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.26
2008-01-16,0.9572,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,4.22
2008-01-23,0.9574,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,3.43
2008-01-30,0.9579,13.5,10.64,12.81,14.4,91.9,2174.5,891.0,12.175,1979.35,...,18863.43405,25.89,1127.98,11.03,10.72,3336.9,-417.88,16843.003,87.093,3.26


In [14]:
def make_supervised(data: pd.DataFrame, target: str, horizon: int, lags: List[int], drop_cols_exact: List[str] = None) -> Tuple[pd.DataFrame, pd.Series, List[str]]:
    """
    Builds a supervised dataset at weekly frequency:
      - Features: lagged target (AR terms) + lagged exogenous features
      - Target (levels): y_{t+h}
    Returns (X, y, feature_names)
    """
    drop_cols_exact = drop_cols_exact or []
    exogenous_columns = [c for c in data.columns if c !=
                         target and c not in drop_cols_exact]

    work = data.copy()

    # Lag target (AR terms)
    for L in lags:
        work[f"{target}_L{L}"] = work[target].shift(L)

    # Lag all exogenous features
    for column in exogenous_columns:
        for L in lags:
            work[f"{column}_L{L}"] = work[column].shift(L)

    # Supervised target for LEVELS (align ahead)
    work[f"{target}_future_h{horizon}"] = work[target].shift(-horizon)

    # Drop rows with any NaNs created by lagging / shifting
    feat_cols = [c for c in work.columns if c.endswith(
        tuple([f"_L{L}" for L in lags]))]
    x = work[feat_cols].copy()
    y = work[f"{target}_future_h{horizon}"].copy()

    valid = x.notna().all(axis=1) & y.notna()
    x, y = x.loc[valid], y.loc[valid]

    return x, y, feat_cols

In [15]:
def make_supervised_returns(data: pd.DataFrame, target: str, horizon: int, lags: List[int], drop_cols_exact: List[str] = None) -> Tuple[pd.DataFrame, pd.Series, pd.Series, List[str]]:
    """
    Supervised dataset for RETURNS:
      - y_ret_h = log(y_{t+h}) - log(y_t)
      - Features: lagged target level & return, lagged exogenous
    Also returns y_level_t (for reconstruction to levels).
    """
    drop_cols_exact = drop_cols_exact or []
    exogenous_columns = [c for c in data.columns if c !=
                 target and c not in drop_cols_exact]

    work = data.copy()
    work["log_y"] = np.log(work[target])
    work["ret1"] = work["log_y"].diff(1)

    # Lag target level & return
    for L in lags:
        work[f"{target}_L{L}"] = work[target].shift(L)
        work[f"ret1_L{L}"] = work["ret1"].shift(L)

    # Lag exogenous
    for column in exogenous_columns:
        for L in lags:
            work[f"{column}_L{L}"] = work[column].shift(L)

    # Future h-step log-return
    work[f"ret_h{horizon}"] = work["log_y"].shift(-horizon) - work["log_y"]

    feat_cols = [c for c in work.columns if c.endswith(
        tuple([f"_L{L}" for L in lags]))]
    x = work[feat_cols].copy()
    y = work[f"ret_h{horizon}"].copy()
    y_level_t = work[target].copy()  # to reconstruct levels later

    valid = x.notna().all(axis=1) & y.notna() & y_level_t.notna()
    x, y, y_level_t = x.loc[valid], y.loc[valid], y_level_t.loc[valid]

    return x, y, y_level_t, feat_cols

In [16]:
# ===========================
# 4) CV Splitter (Expanding Window)
# ===========================
def expanding_splits_index(n_obs: int, n_splits: int, min_train_frac=0.6):
    """
    Generate expanding-window splits over index range [0..n_obs-1].
    Ensures first train fold covers ~min_train_frac of data.
    """
    min_train = int(n_obs * min_train_frac)
    fold_sizes = (n_obs - min_train) // n_splits
    starts = [min_train + i*fold_sizes for i in range(n_splits)]
    ends = starts[1:] + [n_obs]
    for s, e in zip(starts, ends):
        train_index = np.arange(0, s)
        test_index = np.arange(s, e)
        yield train_index, test_index

In [17]:
# ===========================
# 5) Metrics & Helpers
# ===========================
def safe_mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    denom = np.maximum(np.abs(y_true), eps)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100.0)

In [18]:
def direction_accuracy(y_true, y_pred):
    return float(np.mean(np.sign(np.diff(y_true)) == np.sign(np.diff(y_pred))) * 100.0)

In [19]:
def summarize_metrics(y_true, y_pred) -> Dict[str, float]:
    return dict(
        R2=r2_score(y_true, y_pred),
        MAE=mean_absolute_error(y_true, y_pred),
        RMSE=math.sqrt(mean_squared_error(y_true, y_pred)),
        MAPE=safe_mape(y_true, y_pred),
        DirectionalAccuracy=direction_accuracy(y_true, y_pred)
    )

In [20]:
# ===========================
# 6) SARIMAX Backtest (levels)
#    - Fit on train, dynamic forecast the entire test window
#    - Exogenous features are lagged (built in make_supervised), but SARIMAX expects aligned exog.
#      We'll pass current-period exog; lags are columns in X.
# ===========================
def back_test_sarimax_levels(df_weekly, horizon, lags, order=(1, 1, 1), seasonal_order=(1, 0, 0, 52)):
    # Build supervised for levels (X has only lagged features)
    drop_cols = []  # we already removed leakage
    x_all, y_all, feat_cols = make_supervised(
        df_weekly, TARGET_COL, horizon, lags, drop_cols)

    index_all = y_all.index
    y_hat_all = pd.Series(index=index_all, dtype=float)

    # Align an "exog matrix" for SARIMAX equal to feat_cols (lagged features)
    all_exogenous= x_all.copy()

    # Expanding folds on the supervised index
    for train_index, test_index in expanding_splits_index(len(index_all), N_SPLITS, min_train_frac=0.6):
        train_dates = index_all[train_index]
        test_dates = index_all[test_index]

        y_train, y_test = y_all.loc[train_dates], y_all.loc[test_dates]
        exogenous_train, exogenous_test = all_exogenous.loc[
            train_dates], all_exogenous.loc[test_dates]

        # Fit SARIMAX on TRAIN target (levels) using lagged features as exog
        model = SARIMAX(
            endog=y_train, exog=exogenous_train,
            order=order, seasonal_order=seasonal_order,
            enforce_stationarity=False, enforce_invertibility=False, trend='c'
        )
        results = model.fit(disp=False)

        # Dynamic forecast over TEST window using TEST exog
        pred = results.get_forecast(steps=len(y_test), exog=exogenous_test)
        y_hat = pred.predicted_mean
        y_hat_all.loc[y_test.index] = y_hat.values

    # Metrics on the combined OOS forecasts
    metrics = summarize_metrics(y_all.values, y_hat_all.values)
    return y_all, y_hat_all, metrics, feat_cols

In [21]:
# ===========================
# 7) XGBoost Backtest (returns -> reconstruct levels for R² on levels)
# ===========================
def back_test_xgb_returns_to_levels(df_weekly, horizon, lags, xgb_params=None):
    xgb_params = xgb_params or dict(
        n_estimators=800,
        learning_rate=0.03,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=2.0,
        random_state=RANDOM_SEED
    )

    X_all, y_ret_all, y_level_t_all, feat_cols = make_supervised_returns(
        df_weekly, TARGET_COL, horizon, lags, drop_cols_exact=[]
    )

    idx_all = y_ret_all.index
    y_hat_level_all = pd.Series(index=idx_all, dtype=float)

    # Expanding folds on supervised index
    for tr_idx, te_idx in expanding_splits_index(len(idx_all), N_SPLITS, min_train_frac=0.6):
        tr_dates = idx_all[tr_idx]
        te_dates = idx_all[te_idx]

        X_tr, X_te = X_all.loc[tr_dates], X_all.loc[te_dates]
        y_tr, y_te = y_ret_all.loc[tr_dates], y_ret_all.loc[te_dates]
        y_level_t_tr = y_level_t_all.loc[tr_dates]
        y_level_t_te = y_level_t_all.loc[te_dates]

        mdl = XGBRegressor(**xgb_params)
        mdl.fit(X_tr, y_tr)

        # Predict h-step returns and reconstruct to levels:
        # y_hat_{t+h} = y_t * exp(pred_ret_h)
        ret_hat = mdl.predict(X_te)
        y_hat_lvl = y_level_t_te * np.exp(ret_hat)
        y_hat_level_all.loc[te_dates] = y_hat_lvl

    # True future levels aligned to index (built in make_supervised_returns)
    y_true_level = df_weekly[TARGET_COL].shift(-horizon).reindex(idx_all)

    # Metrics on levels
    metrics = summarize_metrics(y_true_level.values, y_hat_level_all.values)
    return y_true_level, y_hat_level_all, metrics, feat_cols

In [22]:
df_weekly.isnull().sum()

Buying                 0
MPR                    0
FoodInflation          0
GhInflationRate        0
NonFoodInflation       0
BrentOil               0
Cocoa                  0
Gold                   0
GhInterestRate         0
NetForeignAssets       0
NIR                    0
GhGDP                  0
GhCompositeIndex       0
FXDeposits             0
T-bill-182             0
T-bill-91              0
PrivateSectorCredit    0
TradeBalance           0
USGDP                  0
USInflationRate        0
USInterestRate         0
dtype: int64

In [23]:
# ===========================
# 8) Run Backtests: SARIMAX (levels) & XGBoost (returns->levels)
# ===========================
all_metrics = []
results_store = {}  # for plotting/SHAP later

for h in HORIZONS:
    # SARIMAX
    # Drop NaNs before backtest (needed by scikit-learn utils inside SARIMAX)
    # Restrict to only target and feature columns needed by SARIMAX, then drop NaNs only from these columns
    sarimax_cols = ["Buying", "MPR", "FoodInflation", "GhInflationRate", "NonFoodInflation", "BrentOil",
                    "Cocoa", "Gold", "GhInterestRate", "NetForeignAssets", "NIR", "GhGDP", "GhCompositeIndex",
                    "FXDeposits", "T-bill-182", "T-bill-91", "PrivateSectorCredit", "TradeBalance", "USGDP",
                    "USInflationRate", "USInterestRate"]

    df_sarimax = df_weekly[sarimax_cols].dropna()
    y_true_sar, y_hat_sar, m_sar, feat_sar = back_test_sarimax_levels(
        df_sarimax, horizon=h, lags=LAGS,
        order=(1, 1, 1), seasonal_order=(1, 0, 0, 52)
    )
    m_sar.update(dict(Model="SARIMAX", Horizon=h))
    all_metrics.append(m_sar)
    results_store[("SARIMAX", h)] = (y_true_sar, y_hat_sar, feat_sar)

    # XGBoost
    y_true_xgb, y_hat_xgb, m_xgb, feat_xgb = back_test_xgb_returns_to_levels(
        df_no_na, horizon=h, lags=LAGS
    )
    m_xgb.update(dict(Model="XGBoost", Horizon=h))
    all_metrics.append(m_xgb)
    results_store[("XGBoost", h)] = (y_true_xgb, y_hat_xgb, feat_xgb)

metrics_df = pd.DataFrame(all_metrics).set_index(
    ["Model", "Horizon"]).sort_index()
metrics_df


KeyboardInterrupt: 